## Build Functions for ELT

## Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 uninstall pandas pyarrow fireducks numpy -y
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime

import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppresses INFO and WARNING messages
import tensorflow as tf

from src import modules as f # f is for function
%load_ext autoreload
%autoreload 2

# Check if GPU is available
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

E0000 00:00:1733826811.208526   27407 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733826811.211676   27407 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num GPUs Available: 1


### Download, Transform, and Modeling All in One

In [15]:
f.predictions('mrvl')

MRVL 5m Interval Timestamp: 2024-12-15 15:40:26 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,static
kelly_1:2.5,0.190025,0.132494,0.18465,0.109091
prob_up,0.437246,0.43741,0.444595,0.375
prob_static,0.384365,0.336314,0.312926,0.5
prob_down,0.178388,0.226276,0.242479,0.125
precision,0.421446,0.380353,0.417607,0.363636
recall,0.411192,0.367397,0.450122,0.517073
f1,0.416256,0.373762,0.433255,0.426989
support,"[411.0, 410.0, 412.0]","[411.0, 410.0, 412.0]","[411.0, 410.0, 412.0]","[411.0, 410.0, 412.0]"


MRVL 15m Interval Timestamp: 2024-12-15 15:40:29 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,up,static
kelly_1:2.5,0.136364,0.187097,0.131325,0.092593
prob_up,0.348383,0.356611,0.377977,0.375
prob_static,0.40132,0.408492,0.347739,0.5
prob_down,0.250297,0.234897,0.274285,0.125
precision,0.383117,0.419355,0.379518,0.351852
recall,0.385621,0.424837,0.409091,0.496732
f1,0.384365,0.422078,0.39375,0.411924
support,"[154.0, 153.0, 153.0]","[154.0, 153.0, 153.0]","[154.0, 153.0, 153.0]","[154.0, 153.0, 153.0]"


MRVL 1h Interval Timestamp: 2024-12-15 15:40:30 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,static,down
kelly_1:2.5,0.127924,0.154922,0.145859,0.07027
prob_up,0.345156,0.383803,0.331303,0.25
prob_static,0.342097,0.307953,0.35731,0.25
prob_down,0.312747,0.308244,0.311388,0.5
precision,0.377088,0.396373,0.389899,0.335907
recall,0.390123,0.377778,0.471883,0.214286
f1,0.383495,0.386852,0.426991,0.261654
support,"[405.0, 409.0, 406.0]","[405.0, 409.0, 406.0]","[405.0, 409.0, 406.0]","[405.0, 409.0, 406.0]"


MRVL 1wk Interval Timestamp: 2024-12-15 15:40:37 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,down
kelly_1:2.5,0.080808,0.170874,0.090722,-0.043636
prob_up,0.313897,0.269188,0.223959,0.25
prob_static,0.571664,0.558294,0.505199,0.25
prob_down,0.114439,0.172518,0.270842,0.5
precision,0.343434,0.407767,0.350515,0.254545
recall,0.404762,0.5,0.404762,0.170732
f1,0.371585,0.449198,0.375691,0.20438
support,"[83.0, 84.0, 82.0]","[83.0, 84.0, 82.0]","[83.0, 84.0, 82.0]","[83.0, 84.0, 82.0]"


MRVL 1mo Interval Timestamp: 2024-12-15 15:40:38 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,up,static,up
kelly_1:2.5,0.3,0.16,0.222222,0.115789
prob_up,0.41141,0.509517,0.33833,0.625
prob_static,0.505759,0.202697,0.383785,0.125
prob_down,0.082831,0.287786,0.277885,0.25
precision,0.5,0.4,0.444444,0.368421
recall,0.421053,0.352941,0.421053,0.411765
f1,0.457143,0.375,0.432432,0.388889
support,"[17.0, 19.0, 17.0]","[17.0, 19.0, 17.0]","[17.0, 19.0, 17.0]","[17.0, 19.0, 17.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')